In [1]:
def retrieve_codes():
    """Convert a csv file full of name and Quandl codes for the Dow Jones top 30 companies,
       as well as the feature codes to a dataframe for further processing in 
       retrieve__data()""" 
    sp500_codes = pd.read_csv("C:/Users/JohnSmith2/Downloads/SP500.csv")
    dow_jones_codes = pd.read_csv("C:/Users/JohnSmith2/Downloads/dowjonesA.csv")
    features_codes = pd.read_csv("C:/Users/JohnSmith2/Downloads/features.csv") 
    nasd_codes = pd.read_csv("C:/Users/JohnSmith2/Downloads/NASDAQComposite.csv")
    nyse_codes = pd.read_csv("C:/Users/JohnSmith2/Downloads/NYSEComposite.csv")
    feat_df = pd.DataFrame(features_codes)
    dj_df = pd.DataFrame(dow_jones_codes)
    sp_df = pd.DataFrame(sp500_codes)
    nasd_df = pd.DataFrame(nasd_codes)
    nyse_df = pd.DataFrame(nyse_codes)
    dj_df = dj_df.ix[:,['name','free_code']]
    sp_df = sp_df.ix[:,['name','free_code']]
    nasd_df = nasd_df.ix[:,['name','free_code']]
    nyse_df = nyse_df.ix[:,['name','free_code']]
    return dj_df, sp_df, feat_df, nasd_df, nyse_df
def retrieve_qunadl_data(dj_df, sp_df, feat_df, nasd_df, nyse_df):
    """Convert both the dow jones and features dataframes of names and codes and download each set
        of data for each name/code pair into a seperate dataframe from the Quandl server. Take all 
        of these dataframes and put them into dictionaries, one for the dow jones info, the other
        for the feature information.""" 
    dowJonesDict,sp500Dict,featuresDict,nasdDict,nyseDict = {},{},{},{},{}   
    start = time.time()
    for name, value in zip(sp_df['name'], sp_df['free_code']):
        each = Quandl.get(value, trim_start="2016-04-15", trim_end="2016-04-15",
                          authtoken="gEC9xAKi4avigPpoQPX1")
        df = pd.DataFrame(each)
        sp500Dict[name] = df
    end = time.time()
    sptime = end - start
    print "sptime", sptime, "seconds for retrieval"    
    start2 = time.time()
    for dow_name, dow_value in zip(dj_df['name'], dj_df['free_code']):
        each_dow = Quandl.get(dow_value, trim_start="2010-04-15", trim_end="2016-04-15", 
                                authtoken="gEC9xAKi4avigPpoQPX1")
        d_df = pd.DataFrame(each_dow)
        dowJonesDict[dow_name] = d_df
    end2 = time.time()
    dowtime = end2 - start2
    print "dowtime", dowtime, "seconds for retrieval"   
    start3 = time.time()
    for feat_name, feat_value in zip(feat_df['name'], feat_df['free_code']):
        each_feat = Quandl.get(feat_value, trim_start="2010-04-15", trim_end="2016-04-15", 
                                authtoken="gEC9xAKi4avigPpoQPX1")
        f_df = pd.DataFrame(each_feat)
        featuresDict[feat_name] = f_df
    end3 = time.time()
    feattime = end3 - start3
    print"feattime", feattime, "seconds for retrieval"   
    i = 0
    for nasd_name, nasd_value in zip(nasd_df['name'], nasd_df['free_code']):
        i += 1
        print i, nasd_name
        each = Quandl.get(nasd_value, trim_start="2016-04-15", trim_end="2016-04-15",
                        authtoken="gEC9xAKi4avigPpoQPX1")
        df = pd.DataFrame(each)
        nasdDict[name] = df
    i = 0
    for nyse_name, nyse_value in zip(nyse_df['name'], nyse_df['free_code']):
        i += 1
        print i, nyse_name
        each = Quandl.get(nyse_value, trim_start="2016-04-15", trim_end="2016-04-15",
                        authtoken="gEC9xAKi4avigPpoQPX1")
        df = pd.DataFrame(each)
        nyseDict[name] = df
    return dowJonesDict, sp500Dict, featuresDict, nasdDict, nyseDict
def create_adj_vol_and_close_dfs(dateAdjDowDict,dateAdjSp500Dict,dateAdjNasdDict,dateAdjNyseDict):
    """Take the dow jones dataframes and create dataframe which contain only the
       adjusted volume and adjusted closing price data. Each dataframe will contain
       all 30 companies going across, and time going down for the past 6 years of 
       data."""
    adjustedDicts,normalDicts = {},{}
    # Make Dow Jones adjusted closing price dataframe with dates as index and
    # companies as columns
    i = True
    for each in dateAdjDowDict.keys():
        if i == True: # signifies first dictionary entry
            df = dateAdjDowDict[each]
            adj_close_df = df[['Adj. Close']].rename(columns={'Adj. Close': each})
            i = False
        else:
            df = dateAdjDowDict[each]
            next_adj_close_df = df[['Adj. Close']].rename(columns={'Adj. Close': each})
            adj_close_df = adj_close_df.join(next_adj_close_df, how='outer')   
    adjustedDicts['adjDowDict'] = adj_close_df
    # Make S&P 500 adjusted closing price dataframe with dates as index and
    # companies as columns
    i = True
    for each in dateAdjSp500Dict.keys():
        if i == True:
            df = dateAdjSp500Dict[each]
            adj_close_df = df[['Adj. Close']].rename(columns={'Adj. Close': each})
            i = False
        else:
            df = dateAdjSp500Dict[each]
            next_adj_close_df = df[['Adj. Close']].rename(columns={'Adj. Close': each})
            adj_close_df = adj_close_df.join(next_adj_close_df, how='outer')  
    adjustedDicts['adjSpDict'] = adj_close_df
    # Make Nasdaq google-finance-styled closing price dataframe with dates as index and
    # companies as columns
    i = True
    for each_norm in dateAdjNasdDict.keys():
        if i == True:
            norm_df = dateAdjNasdDict[each_norm]
            norm_close_df = norm_df[['Close']].rename(columns={'Close': each_norm})
            i = False
        else:
            norm_df = dateAdjNasdDict[each_norm]
            next_norm_close_df = norm_df[['Close']].rename(columns={'Close': each_norm})
            norm_close_df = norm_close_df.join(next_norm_close_df, how='outer')      
    normalDicts['normNasdDict'] = norm_close_df
    # Make Nasdaq google-finance-styled closing price dataframe with dates as index and
    # companies as columns
    i = True
    for each_norm in dateAdjNyseDict.keys():
        if i == True:
            norm_df = dateAdjNyseDict[each_norm]
            norm_close_df = norm_df[['Close']].rename(columns={'Close': each_norm})
            i = False
        else:
            norm_df = dateAdjNyseDict[each_norm]
            next_norm_close_df = norm_df[['Close']].rename(columns={'Close': each_norm})
            norm_close_df = norm_close_df.join(next_norm_close_df, how='outer') 
    normalDicts['normNyseDict'] = norm_close_df      
    return adjustedDicts, normalDicts
def adj_for_stk_record_nums(dowJonesDict, sp500Dict, nasdDict, nyseDict):
    dateAdjNasdDict,dateAdjNyseDict = {},{}
    dateAdjSp500Dict,dateAdjDowDict = {},{}
    for each_key in dowJonesDict.keys():
        length = len(dowJonesDict[each_key])
        value = dowJonesDict[each_key]
        if length > 4000:
            dateAdjDowDict[each_key] = value
    for each_key2 in sp500Dict.keys():
        length2 = len(sp500Dict[each_key2])
        value2 = sp500Dict[each_key2]
        if length2 > 4000:
            dateAdjSp500Dict[each_key2] = value2
    for each_key3 in nasdDict.keys():
        length3 = len(nasdDict[each_key3])
        value3 = nasdDict[each_key3]
        if length3 > 4000:
            dateAdjNasdDict[each_key3] = value3
    for each_key4 in nyseDict.keys():
        length4 = len(nyseDict[each_key4])
        value4 = nyseDict[each_key4]
        if length4 > 4000:
            dateAdjNyseDict[each_key4] = value4
    return dateAdjDowDict, dateAdjSp500Dict, dateAdjNasdDict, dateAdjNyseDict
def fill_in_nulls(adjustedDicts,normalDicts):
    for each_adj_df in adjustedDicts.keys():
        adj_df = adjustedDicts[each_adj_df]
        ifnull = adj_df.isnull().values.any()
        if ifnull == True:
            adj_df.fillna(method="ffill",inplace="TRUE")
            adj_df.fillna(method="bfill",inplace="TRUE")
    for each_norm_df in normalDicts.keys():
        norm_df = normalDicts[each_norm_df]
        ifnull = norm_df.isnull().values.any()
        if ifnull == True:
            norm_df.fillna(method="ffill",inplace="TRUE")
            norm_df.fillna(method="bfill",inplace="TRUE")
    return adjustedDicts, normalDicts

dowJonesDict, sp500Dict, featuresDict, nasdDict, nyseDict = retrieve_quandl_data(dj_df, sp_df, feat_df, nasd_df, nyse_df)
dateAdjDowDict, dateAdjSp500Dict, dateAdjNasdDict, dateAdjNyseDict = adj_for_stk_record_nums(
    dowJonesDict, sp500Dict, nasdDict, nyseDict)
adjustedDicts, normalDicts = create_adj_vol_and_close_dfs(
    dateAdjDowDict, dateAdjSp500Dict, dateAdjNasdDict, dateAdjNyseDict)
adjustedDicts, normalDicts = fill_in_nulls(adjustedDicts,normalDicts)
print "Done"

Done


In [3]:
def compute_bollinger_bands(adjCloseDict, window):
    """Take the adjusted closing prices dataframe and compute the bollinger bands for each company,
       using 20 day windows for now. Will adjust the window date as needed. Computes both the upper
       and lower bands and stores them into dataframes.
       Note, first window is blank so you need to adjust for that by using data from 20 days forward."""
    eachIndexBolBandsDict = {}
    for each_key in adjCloseDict.keys():
        each_index = adjCloseDict[each_key]
        i = True
        for each in each_index.keys():
            rm_company = get_rolling_mean(each_index[each], window)
            rstd_company = get_rolling_std(each_index[each], window)
            upper_band, lower_band = get_bollinger_bands(rm_company, rstd_company)
            if i == True:
                upper_band_df = pd.DataFrame(upper_band)
                lower_band_df = pd.DataFrame(lower_band)
                rm_company_df = pd.DataFrame(rm_company)
                i = False
            else:
                next_upper_df = pd.DataFrame(upper_band)
                next_lower_df = pd.DataFrame(lower_band)
                next_rm_df    = pd.DataFrame(rm_company)

                upper_band_df = upper_band_df.join(next_upper_df, how='outer')
                lower_band_df = lower_band_df.join(next_lower_df, how='outer')
                rm_company_df = rm_company_df.join(next_rm_df, how='outer')
        eachIndexBolBandsDict[each_key] = [upper_band_df, lower_band_df, rm_company_df]
    return eachIndexBolBandsDict
def get_momentum(adjCloseDict, window):
    i,y = 0,True
    temp_array = []
    eachIndexMomentumDict = {}
    for each_dict in adjCloseDict:
        for each_key in each_dict.keys():
            each_index = each_dict[each_key]
            for each_company in each_index.keys():
                each_column = each_index[each_company]
                each_column = each_column.as_matrix()
                for each in xrange(len(each_column)):
                    if i > window:
                        temp = each_column[i]/each_column[i-window] - 1
                        temp_array.append(temp)
                    else:
                        temp_array.append(0.)
                    i += 1
                if y == True:
                    df = pd.DataFrame(temp_array, index=each_index.index, columns=[each_company])
                    y = False
                else:
                    df2 = pd.DataFrame(temp_array, index=each_index.index, columns=[each_company])
                    df = df.join(df2, how='outer')
                temp_array = []
                i = 0
            y = True
            eachIndexMomentumDict[each_key] = df
    return eachIndexMomentumDict
def pe_ratio(adjCloseDict, window):
    i,y = 0,True
    pe_array = []
    eachIndexPeRatioDict = {}
    for each_key in adjCloseDict.keys():
        each_index = adjCloseDict[each_key]
        for each_comp in each_index.keys():
            each_column = each_index[each_comp]
            each_column = each_column.as_matrix()
            for each in xrange(len(each_column)):
                if i > window:
                    each_return = each_column[i] - each_column[i-window]
                    each_pe = each_column[i]/each_return
                    pe_array.append(each_pe)
                else:
                    pe_array.append(0.)
                i += 1
            if y == True:
                pe_df = pd.DataFrame(pe_array, index=each_index.index, columns=[each_comp])
                y = False
            else:
                pe_df2 = pd.DataFrame(pe_array, index=each_index.index, columns=[each_comp])
                pe_df = pe_df.join(pe_df2, how='outer')
            pe_array = []
            i = 0
        y = True
        eachIndexPeRatioDict[each_key] = pe_df
    return eachIndexPeRatioDict
def get_rolling_mean(values, window):
    """Return rolling mean of given values, using specified window size."""
    return pd.rolling_mean(values, window=window)
def get_rolling_std(values, window):
    """Return rolling standard deviation of given values, using specified window size"""
    return pd.rolling_std(values, window=window)   
def get_bollinger_bands(rm, rstd):
    """Return upper and lower Bollinger Bands."""
    upper_band = rm + rstd * 2
    lower_band = rm - rstd * 2
    return upper_band, lower_band
def normalize_data(df_dict):
    """Normalize the adjusted_closing_price dataframes"""
    normalize = []
    for each in df_dict:
        norm_df =  each / each.ix[0,:]
        normalize.append(norm_df)
    return normalize

eachIndexBolBandsDict = compute_bollinger_bands(adjusted_dict_array[1], 20)
eachIndexMomentumDict = get_momentum(adjusted_dict_array[1], 10)
eachIndexPeRatioDict = pe_ratio(adjusted_dict_array[1], 20)
print "DONE"

In [53]:
def plot_data(df, title="Stock prices", xlabel="Date", ylabel="Price"):
    """Plot stock prices with a custom title and meaningful axis labels."""
    ax = df.plot(title=title, fontsize=12)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.show()
def plot_bollinger(adj_close_df):
    """Takes the rolling means and the upper and lower bands and plots the data. Currently using only
       a 140 day graph, but can easily change to show less or more information by changing the adjusting
       closing price dataframe."""
    rm_SPY = get_rolling_mean(adj_close_df, window=20)
    rstd_SPY = get_rolling_std(adj_close_df, window=20)
    upper_band, lower_band = get_bollinger_bands(rm_SPY, rstd_SPY)
    # Plot raw SPY values, rolling mean and Bollinger Bands
    ax = adj_close_df[20:160].plot(title="Bollinger Bands", label='IBM')
    rm_SPY[20:].plot(label='Rolling mean', ax=ax)
    upper_band[20:].plot(label='upper band', ax=ax)
    lower_band[20:].plot(label='lower band', ax=ax)
    # Add axis labels and legend
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.legend(loc='upper left')
    plt.show()     
def compute_and_plot_daily_returns(adj_close_df):
    """Compute the daily return values for each company by calling compute_daily_returns() and then
       plot the returned values for a 20 day period. This 20 day period can be adjusted as necessary
       by increasing or decreasing the size of the adj_close_df when this function is called. """
    #Compute daily returns
    daily_returns = compute_daily_returns(adj_close_df)
    plot_data(daily_returns, title="Daily returns", ylabel="Daily returns")
    
plot_bollinger(adjusted_dict_array[0]['adjDowDict'][['IBM']][:160])
compute_and_plot_daily_returns(adjusted_dict_array[0]['adjDowDict'][0:20])
print "Done"

In [54]:
#buy/sell/nothing == actions
#holding/shorting/bollinger/momentum/pe/rewardsinceentry == state
#dailyreturn == reward

Done
52582
11149
436269
